In [ ]:
user_input = """나경민의 주루사 때 카메라에 잡혔던 어느 롯데 팬의 절규가 큰 화제가 되었다.[14] 모래반지 빵야빵야처럼 파레이돌리아가 가능하다.

당시 나온 드립만 해도 가지각색이다.
- 야

이후 저 팬은 나경민에게 사인 유니폼을 받았다고 한다. 게다가 옷에 달고 있던 TWICE의 두 번째 투어 콘서트 'TWICELAND ZONE 2 : Fantasy Park'의 공식 굿즈인 캐릭터 핀 버튼[18]을 유니폼에 달고 있던 모습 때문에 ONCE들 사이에서도 유명세를 탔다.
"""
output = """없음
"""

In [ ]:
instruct = (
    "You are a Table of Contents extractor. "
    "User will speak to you questions. "
    "You must reply only with [목차(Table of Contents)] part extracted from the questions. "
    "You must keep original text. Do not change original text."
    "And you must not involve [dotted line, page number, 제목(title), content, explanation, summary, predicted]. "
    "do not write explanations."
)

In [ ]:
from fastchat.model.model_adapter import get_conversation_template
conv = get_conversation_template("chat-orca")

In [ ]:
"""
split chunks
"""

text = ori_text#.replace('\n', '')
matches = re.finditer(r'\n', text)
print("text length:", len(text))

max_chunk_length = 2048
result_chunks = []

left = 0
right = 0
match = -1
while(match is not None):
    try:
        match = next(matches)
        midx = match.start()
    except:
        match = None
        midx = len(text)
    
    if midx - left < max_chunk_length:
        right = midx
    else:
        # text split is longer than max_chuck_length
        if left >= right:
            while(midx - left >= max_chunk_length):
                chunk = text[left:left+max_chunk_length]
                left += max_chunk_length
                right += max_chunk_length
                result_chunks.append(chunk)
            right = midx
        chunk = text[left:right+1]
        left = right+1
        right = midx
        result_chunks.append(chunk)
        
if left < right: # get last split
    chunk = text[left:right+1]
    result_chunks.append(chunk)
    
print("num chunk:", len(result_chunks))

In [ ]:
%%time
"""
extract ToC
"""

api_server_url = "http://localhost:21122"
instruct = (
    "You are a Table of Contents extractor. "
    "User will speak to you questions. "
    "You must reply only with [목차(Table of Contents)] part extracted from the questions. "
    "You must keep original text. Do not change original text. "
    "And you must not involve [dotted line, page number, 제목(title), 참고문헌(reference), 부록(appendix), content, explanation, summary, predicted]. "
    "When there is no Table of Contents, you must reply with \"없음\". "
    "do not write explanations."
)
prompt_template = """### System:
{instruct}
### User: {data}

### Assistant:"""
def send_extract_toc(tocs):
    global idx
    # for _ in range(2):
    while(1):
        if idx > len_data - 1:
            break
        lock.acquire()
        pidx = idx
        data = result_chunks[pidx]
        idx += 1
        lock.release()
        
        print(f"{idx}/{len_data}", '\t\t\t\t\t\t', end='\r')#
        
        # response
        prompt = prompt_template.format(
            instruct=instruct,
            data=data
        )
        input_json = {
            "model_name": "MDIEM-toc3", #"OLLM-Small_2024.01.16", #"MDIEM-toc3",
            "prompt": prompt,
            "temperature": 0.7,
            "top_p": 0.8,
            "max_new_tokens": 512,
        }

        ret = requests.post(
            api_server_url + "/worker_generate",
            json=input_json,
            timeout=30,
        )

        result = ret.json()['text'][len(input_json['prompt'])+1:]
        tocs[pidx] = result

tocs = [""] * len(result_chunks)
threads = []
idx = 0
lock = threading.Lock()
len_data = len(result_chunks)
n_thread = 16

for i in range(n_thread):
    t = threading.Thread(target=send_extract_toc, args=(tocs,)) # 
    t.start()
    threads.append(t)
    
for t in threads:
    t.join()
    
    
first_toc = '\n'.join(tocs)
# response
prompt = prompt_template.format(
    instruct=instruct,
    data=first_toc
)
input_json = {
    "model_name": "MDIEM-toc3", #"OLLM-Small_2024.01.16", # 
    "prompt": prompt,
    "temperature": 0.7,
    "top_p": 0.8,
    "max_new_tokens": 1024,
}

ret = requests.post(
    api_server_url + "/worker_generate",
    json=input_json,
    timeout=30,
)

second_toc = ret.json()['text'][len(input_json['prompt'])+1:]
print(second_toc)

In [ ]:
print(first_toc)

In [ ]:
"""
make ToC-Contents Groups
"""

toc_string = re.sub(r'\n+', '\n', second_toc)
parts = toc_string.split('\n')

MIN_SPLIT_LEN = 1

def remove_empty(parts):
    new_parts = []
    for part in parts:
        if len(part) <= MIN_SPLIT_LEN: continue
        new_parts.append(part)
    return new_parts

parts = remove_empty(parts)

parts_indexs = []
for part in parts:
    matched = [match.start() for match in re.finditer(part, ori_text)]
    for match in matched:
        parts_indexs.append((part, match))
    print(part, matched)
    
parts_indexs = sorted(parts_indexs, key=lambda x: x[1])
print(parts_indexs)

toc_dict = {}
for pdata1, pdata2 in zip(parts_indexs, parts_indexs[1:] + [('', len(ori_text)-1)]):
    name, left = pdata1
    left += len(name)
    _, right = pdata2
    
    if right - left > MIN_SPLIT_LEN:
        if name not in toc_dict:
            toc_dict[name] = [ori_text[left:right]]
        else:
            toc_dict[name].append(ori_text[left:right])

In [ ]:
%%time
"""
Summary ToC-Contents Groups
"""

MIN_CONTENT_LEN = 10

api_server_url = "http://localhost:21122"
# I want you to act as a summaryist. I will give you sentences. "
# f"I want you to only reply with a summarization based on the sentences. do not write explanations.
# "do not write explanations. summarize the sentences as briefly as possible: {query}"
instruct = (
    "do not write explanations. 다음 문장을 한글로 번역하고 최대한 짧게 요약해주세요: {query}"
)
prompt_template = """### System:
This is a system prompt, please behave and help the user.

### User: {instruct}

### Assistant:"""
def send_summary(summaries):
    global idx
    # for _ in range(2):
    while(1):
        if idx > len_data - 1:
            break
        lock.acquire()
        pidx = idx
        data = toc_dict.get(parts[pidx])
        idx += 1
        lock.release()
        
        print(f"{idx}/{len_data}", '\t\t\t\t\t\t', end='\r')#
        
        if data is None:
            continue
        data = '\n'.join(data)
        if len(data) < MIN_CONTENT_LEN:
            # summaries[pidx] = data
            continue
        
        # response
        prompt = prompt_template.format(
            instruct=instruct.format(query=data),
        )
        input_json = {
            "model_name": "OLLM-Large_2023.11.20",# "MDIEM-toc3",
            "prompt": prompt,
            "temperature": 0.8,
            "top_p": 0.7,
            "max_new_tokens": 1024,
        }

        ret = requests.post(
            api_server_url + "/worker_generate",
            json=input_json,
            timeout=300,
        )

        result = ret.json()['text'][len(input_json['prompt'])+1:]
        summaries[pidx] = result

summaries = [""] * len(parts)
threads = []
idx = 0
lock = threading.Lock()
len_data = len(parts)
n_thread = 16

for i in range(n_thread):
    t = threading.Thread(target=send_summary, args=(summaries,)) # 
    t.start()
    threads.append(t)
    
for t in threads:
    t.join()

In [ ]:
for idx in range(len(parts)):
    print(parts[idx], '\n', summaries[idx], '\n')

In [ ]:
for part in parts:
    if part in toc_dict:
        print(part, '\n', len(toc_dict[part]), '\n', toc_dict[part], '\n')